# Setup

In [12]:
# Import necessary libraries
import numpy as np
import math

# Define parameters
demand = np.array([1000, 300, 100, 50])
common_ordering_cost = 100
specific_ordering_cost = np.array([10, 20, 25, 25])
unit_cost = np.array([50, 60, 30, 30])
holding_cost_rate = 0.2

# Scenario 1: Products are sourced independently

In [13]:
# Calculate EOQ and total cost for each product
n_1 = [0,0,0,0]
total_cost_individual = 0
for i in range(len(demand)):
    Q = math.ceil(np.sqrt((2 * demand[i] * (common_ordering_cost + specific_ordering_cost[i])) / (holding_cost_rate * unit_cost[i])))
    n_1[i] = demand[i] / Q
    total_cost_individual += ((unit_cost[i] * holding_cost_rate * Q) / 2) + (((common_ordering_cost + specific_ordering_cost[i]) * demand[i])/Q)
    print("Optimal Q for Product " + str(i) + ":", Q)

# Print the result
print(f"\nAnnual Operational Cost for Scenario 1: ${total_cost_individual:.2f}")

Optimal Q for Product 0: 149
Optimal Q for Product 1: 78
Optimal Q for Product 2: 65
Optimal Q for Product 3: 46

Annual Operational Cost for Scenario 1: $3073.97


# Scenario 2: All four products are sourced with the same frequency

In [14]:
# Compute S
S = common_ordering_cost + sum(specific_ordering_cost)
print("S =", S)

# Compute n
top = np.sum(demand * holding_cost_rate * unit_cost)
bottom = 2*S
n = np.sqrt(top/bottom)
print("n =", n)

# Compute Q for each of the 4 products
Q = [0, 0, 0, 0]
for i in range(len(demand)):
    Q[i] = math.ceil(demand[i] / n)
print("Q =", Q)

# Compute Annual Ordering Cost
# annual ordering cost  = S * n
annual_order_cost = S * n
print(f"\nAnnual Order Cost: ${annual_order_cost:.2f}")

# Compute Annual Holding Cost
# annual holding cost is same formula as part 1 (optimal Q * holding rate * unit cost) / 2
annual_holding_cost = np.sum(((unit_cost * holding_cost_rate * Q) / 2))
print(f"Annual Holding Cost: ${annual_holding_cost:.2f}")


# Compute Total Cost
total_cost_combined = annual_order_cost + annual_holding_cost

# Print the result
print(f"\nAnnual Operational Cost for Scenario 2: ${total_cost_combined:.2f}")

S = 180
n = 6.346477588219924
Q = [158, 48, 16, 8]

Annual Order Cost: $1142.37
Annual Holding Cost: $1150.00

Annual Operational Cost for Scenario 2: $2292.37


# Scenario 3: Order frequencies are determined according to the tailored aggregation strategy

In [15]:
# Step 1: Identify most frequently ordered product:
most_freq_ordered = n_1.index(max(n_1))
print("Most frequently ordered product number:", most_freq_ordered)

Most frequently ordered product number: 0


In [16]:
# Step 2: For all products other than the most frequently ordered, recompute frequencies using ONLY product specific ordering costs
# Compute bottom of the n formula for each product
bottom = np.sum(specific_ordering_cost) - specific_ordering_cost[most_freq_ordered]

# Compute top of the n formula for each product
top = [0,0,0,0]
for i in range(len(top)):
    top[i] = demand[i] * holding_cost_rate * unit_cost[i]

# Compute n for each product
n = [0,0,0,0]
for i in range(len(n)):
    n[i] = np.sqrt(top[i]/bottom)
n[most_freq_ordered] = n_1[most_freq_ordered]
print("n =", n)

n = [6.7114093959731544, 7.171371656006362, 2.9277002188455996, 2.0701966780270626]


In [17]:
# Step 3: Identify frequency of other products as a multiple mi
# Compute Q for each of the 4 products
Q = [0, 0, 0, 0]
for i in range(len(demand)):
    Q[i] = math.ceil(demand[i] / n[i])
print("Q =", Q)

# Frequencies
mi = [0,0,0,0]
for i in range(len(mi)):
    mi[i] = math.ceil(n[most_freq_ordered]/n[i])
print("mi:", mi)

Q = [149, 42, 35, 25]
mi: [1, 1, 3, 4]


In [18]:
# Step 4: Recalcluate ordering frequency n of most frequently ordered product

# Find baseline freq
top_new = np.sum(holding_cost_rate * unit_cost * demand * mi)
bottom_new = 2 * (common_ordering_cost + np.sum(specific_ordering_cost/mi))
baseline_freq = math.sqrt(top_new/bottom_new)
print("Baseline ordering Frequency:", baseline_freq)

# Baseline frequency is the recalculated n of the most frequently ordered product

Baseline ordering Frequency: 7.576697173415134


In [19]:
# Step 5: Identify ordering frequency of all products using n (baseline frequency) and mi

# Find optimal freq of all products
ordering_freq = [0,0,0,0]
for i in range(len(ordering_freq)):
    ordering_freq[i] = baseline_freq / mi[i]
print("Optimal Frequencies:", ordering_freq)

Optimal Frequencies: [7.576697173415134, 7.576697173415134, 2.5255657244717113, 1.8941742933537835]


In [20]:
# Find optimal Q
# demand[i] / demand freq
optimal_q = [0,0,0,0]
for i in range(len(optimal_q)):
    optimal_q[i] = math.ceil(demand[i] / ordering_freq[i])

# Compute Annual Holding Cost
annual_holding_cost = [0,0,0,0]
for i in range(4):
    annual_holding_cost[i] = (unit_cost[i] * holding_cost_rate * optimal_q[i]) / 2
total_annual_holding_cost = np.sum(annual_holding_cost)

# Compute Annual Ordering Cost
# baseline frequency * (common ordering cost + sum product of (specific ordering cost / frequency multiple))
annual_order_cost = baseline_freq * (common_ordering_cost + np.sum(specific_ordering_cost / mi))

# Compute Total Cost
total_cost_combined = annual_order_cost + total_annual_holding_cost

In [21]:
# Print table headers
print("Product Number\tOptimal Order Quantity\tAnnual Holding Cost")

# Iterate over the indices and print corresponding values
for i in range(4):
    print(f"{i}\t\t\t{optimal_q[i]}\t\t\t{annual_holding_cost[i]}")

# Print Holding and Order Totals
print(f"\nTotal Annual Holding Cost: ${total_annual_holding_cost:.2f}")
print(f"Total Annual Order Cost: ${annual_order_cost:.2f}")

# Print the result
print(f"\nAnnual Operational Cost for Scenario 3: ${total_cost_combined:.2f}")

Product Number	Optimal Order Quantity	Annual Holding Cost
0			132			660.0
1			40			240.0
2			40			120.0
3			27			81.0

Total Annual Holding Cost: $1101.00
Total Annual Order Cost: $1095.46

Annual Operational Cost for Scenario 3: $2196.46
